In [176]:
# RSA simple implementing
# 
# 质数
#   只能被1和自身整除
#
# 互质
#   最大公约数为1的数
#
# 欧拉函数
#   φ(pq) = φ(p)φ(q) = (p - 1)(q - 1)
#   r = φ(pq) = lcm(p - 1, q - 1)
#
# 模反元素
#   如果两个正整数a和n互质，那么一定可以找到整数b，使得 ab-1 被n整除，或者说ab被n除的余数是1
#   ab ≡ 1(mod n)  
#   ed ≡ 1(mod r)
#
# 欧拉定理
#   如果两个正整数a和n互质，则n的欧拉函数 φ(n) 可以让下面的等式成立：
#   a^φ(n) ≡ 1(mod n)
#   由此可得：a的φ(n - 1)次方肯定是a关于n的模反元素
#   欧拉定理就可以用来证明模反元素必然存在
#
# 小费马定理
#   假设正整数a与质数p互质，因为质数p的φ(p)等于p-1，则欧拉定理可以写成
#   a^(p-1) ≡ 1 (mod p)
#   这其实是欧拉定理的一个特例
#
#
# RSA generator steps
# step1: p, q : two different primes
#   n = p * q  : modulus
# step2:
#   φ(pq) = φ(p)φ(q) = (p - 1) * (q - 1)
#   r = φ(pq) = lcm(p - 1, q - 1)
# step3: 
#   e * d ≡ 1(mod r) -> e * d % r = 1 
#   e: 1 < e < r and gdc(e, r) == 1
# step4:
#   d: 1 < d < r and d * e mod r == 1
# step5:
#   (e, n) : public key
#   (d, n) : private key
# 
# m: message, m < n
# encrypted = m ** e mod n
# decrypted = encrypted ** d mod n
#
# reference:
#   https://zhuanlan.zhihu.com/p/33580225
#

In [184]:
IGNORE_FIRST_PRIMES_COUNT = 5
P_AND_Q_DVALUE_THRESHOLD = 5
E_AND_D_DVALUE_THRESHOLD = 5

In [185]:
def primes_generator():
    primes = []   # primes generated so far
    last = 1      # last number tried
    while True:
        last += 1
        for p in primes:
            if last % p == 0:
                break
        else:
            primes.append(last)
            yield last

# greatest common divisor
def gcd(x, y):
   while(y):
       x, y = y, x % y
        
   return x

# least common multiple
def lcm(x, y):
   return (x * y) // gcd(x, y)

In [186]:
def get_p_q():
    primes = primes_generator()
    for i in range(1, IGNORE_FIRST_PRIMES_COUNT):
        next(primes)   
    p = next(primes)

    for i in range(1, P_AND_Q_DVALUE_THRESHOLD):
        next(primes)
    q = next(primes)
    
    return (p, q)
        
def get_n(p, q):
    return p * q


def get_r(p, q):
    return lcm(p - 1, q - 1)


def get_e(r):
    for e in range(2, r):
        if gcd(e, r) == 1:
            return e
        else:
            continue
            
    return 0

    
def get_d(r, e):
    for d in range(2, r):
        if d == e and abs(d - e) < E_AND_D_DVALUE_THRESHOLD:
            continue
            
        if e * d % r == 1:
            return d
        else:
            continue
            
    return 0

In [187]:
def get_rsa(p, q):
    rsa = dict()
    rsa['p'] = p
    rsa['q'] = q
    rsa['n'] = p * q
    
    rsa['r'] = get_r(p, q)
    r = rsa['r']
    e = get_e(r)
    if e == 0:
        rsa['ready'] = False
        return rsa
    
    d = get_d(r, e)
    if d == 0:
        rsa['ready'] = False
        return rsa
    
    rsa['e'] = e
    rsa['d'] = d
    rsa['ready'] = True
    
    return rsa

In [188]:
def verify_rsa(rsa, msgs):
    for e, d in (('e', 'd'), ('d', 'e')):
        print("m -> {}(m) -> {}({}(m)) -> m: tesing ...".format(e, d, e))
        for m in msgs:
            m_e = m ** rsa[e] % rsa['n']
            m_e_d = m_e ** rsa[d] % rsa['n']
            print("{}: {} -> {} -> {}".format(m == m_e_d, m, m_e, m_e_d))

In [189]:
def main():
    p, q = get_p_q()
    rsa = get_rsa(p, q)
    print('rsa -> {}'.format(rsa))
    if rsa['ready'] == False:
        print('get RSA failed!')
        return
    msgs = range(1, 20)
    verify_rsa(rsa, msgs)

In [190]:
main()

rsa -> {'p': 11, 'q': 29, 'n': 319, 'r': 140, 'e': 3, 'd': 47, 'ready': True}
m -> e(m) -> d(e(m)) -> m: tesing ...
True: 1 -> 1 -> 1
True: 2 -> 8 -> 2
True: 3 -> 27 -> 3
True: 4 -> 64 -> 4
True: 5 -> 125 -> 5
True: 6 -> 216 -> 6
True: 7 -> 24 -> 7
True: 8 -> 193 -> 8
True: 9 -> 91 -> 9
True: 10 -> 43 -> 10
True: 11 -> 55 -> 11
True: 12 -> 133 -> 12
True: 13 -> 283 -> 13
True: 14 -> 192 -> 14
True: 15 -> 185 -> 15
True: 16 -> 268 -> 16
True: 17 -> 128 -> 17
True: 18 -> 90 -> 18
True: 19 -> 160 -> 19
m -> d(m) -> e(d(m)) -> m: tesing ...
True: 1 -> 1 -> 1
True: 2 -> 84 -> 2
True: 3 -> 163 -> 3
True: 4 -> 38 -> 4
True: 5 -> 80 -> 5
True: 6 -> 294 -> 6
True: 7 -> 248 -> 7
True: 8 -> 2 -> 8
True: 9 -> 92 -> 9
True: 10 -> 21 -> 10
True: 11 -> 44 -> 11
True: 12 -> 133 -> 12
True: 13 -> 238 -> 13
True: 14 -> 97 -> 14
True: 15 -> 280 -> 15
True: 16 -> 168 -> 16
True: 17 -> 41 -> 17
True: 18 -> 72 -> 18
True: 19 -> 211 -> 19
